In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier, cv


import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
df_origin = pd.read_csv("../input/weather-dataset-rattle-package/weatherAUS.csv")

In [ ]:
df_origin.isnull().sum(axis=0) / df_origin.shape[0] * 100

In [ ]:
df_pre = df_origin.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis='columns')

In [ ]:
df_pre['Rainfall'] = df_pre['Rainfall'].fillna(0)

In [ ]:
px.line(df_pre, x="Date", y="Rainfall", color="Location")

In [ ]:
px.line(df_pre, x="Date", y="MaxTemp", color="Location")

In [ ]:
px.line(df_pre, x="Date", y="MinTemp", color="Location")

In [ ]:
df_pre = df_pre.dropna(axis='rows').reset_index(drop=True)

In [ ]:
round(df_pre.shape[0] / df_origin.shape[0], 2)

In [ ]:
Location_dummy = pd.get_dummies(df_pre.Location).astype(float)
WindGustDir_dummy = pd.get_dummies(df_pre.WindGustDir).astype(float)
# WindDir9am_dummy = pd.get_dummies(df_pre.WindDir9am)
# WindDir3pm_dummy = pd.get_dummies(df_pre.WindDir3pm)
RainToday_dummy = pd.get_dummies(df_pre.RainToday).astype(float)

In [ ]:
sc = StandardScaler()
sc.fit(df_pre[['MinTemp', 'MaxTemp', 'Rainfall', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm']])

In [ ]:
df = sc.transform(df_pre[['MinTemp', 'MaxTemp', 'Rainfall', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm']])

In [ ]:
df = pd.DataFrame(df, columns=['MinTemp', 'MaxTemp', 'Rainfall', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm'])

In [ ]:
x_train = pd.concat([Location_dummy, df, WindGustDir_dummy, RainToday_dummy, df_pre.RainTomorrow], axis=1)

In [ ]:
rf = RandomForestClassifier()
rf_param_grid = {'n_estimators' : [5,10,15], 'max_depth' : [5,10]}
rf_grid = GridSearchCV(estimator=rf,
                      param_grid=rf_param_grid,
                      scoring='roc_auc',
                      cv=4,
                      return_train_score=True)

In [ ]:
rf_grid.fit(x_train.iloc[:,:-1], x_train.iloc[:,-1])

In [ ]:
rf_grid_df = pd.DataFrame(rf_grid.cv_results_)
rf_grid_df

In [ ]:
rf_grid_df.loc[rf_grid_df['rank_test_score'] == 1, ]